In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch import nn

In [ ]:
# Initialize GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import numpy as np

X = np.load('../input/sign-language-digits-dataset/X.npy',allow_pickle=True)
y = np.load('../input/sign-language-digits-dataset/Y.npy', allow_pickle=True)

In [ ]:
y.shape

In [ ]:
y = np.argmax(y,1)

In [ ]:
y.shape

In [ ]:
print(np.unique(y, return_counts=True))

In [ ]:
print(X.shape)
#X = np.expand_dims(X, 1)
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(y.mean())

In [ ]:
class CNNDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            self.X = torch.from_numpy(X).unsqueeze(1)
            self.y = torch.from_numpy(y)
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]
      

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1,24, kernel_size=3,padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.2),
            
            nn.Conv2d(24,48,kernel_size=3,padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.2),

            nn.Flatten(),
            nn.Linear(12288, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            
            nn.Linear(1024, 10),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return self.layers(x)

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
cnn = CNN()
summary(cnn.to(device),(1,64,64))

In [ ]:
torch.manual_seed(42)

train_losses = []

dataset = CNNDataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True)

cnn = CNN().to(device)

loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(cnn.parameters(), lr=3*1e-6)

total_train_loss = 0
loss_count = 0

for epoch in range(0, 200):
    current_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        
        #print(targets)
        #print(inputs)

        #targets = targets.reshape((*targets.shape, 1))

        optimizer.zero_grad()

        outputs = cnn(inputs)

        loss = loss_func(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        total_train_loss += loss.item()
        loss_count += 1
        
        if (i+1) % 10 == 0:
            current_loss = 0.0
            
    print(f'Train loss (Crossentropy), epoch {epoch+1}: {total_train_loss/loss_count}')
    train_losses.append(total_train_loss/loss_count)
    loss_list.append(total_train_loss/loss_count)
    loss_count = 0
    total_train_loss = 0

print('Training process has finished.')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.title("CNN training loss")
plt.xlabel("Epoch")
plt.ylabel("Categorical crossentropy Loss")
plt.show()

In [ ]:
# Compute test loss of the model
dataset_test = CNNDataset(X_test, y_test)
testloader = torch.utils.data.DataLoader(dataset_test, batch_size=256, shuffle=True)

total_test_loss = 0
loss_count = 0

# Define the loss function
loss_function = nn.CrossEntropyLoss()

# Run the test loop
cnn.eval()

# Iterate over the DataLoader for training data
for i, data in enumerate(testloader, 0):
    # Get and prepare inputs
    inputs, targets = data
    inputs, targets = inputs.to(device), targets.to(device)

    targets = targets.reshape((targets.shape[0], 1))

    # Perform forward pass
    with torch.no_grad():
        outputs = cnn(inputs)

    # Compute loss
    loss = loss_function(outputs, targets)

    total_test_loss += loss.item()
    loss_count += 1

print(f'Test loss (CE): {total_test_loss/loss_count}')